# Plant Pathology Hypertuning

### Setup

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler

# import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

import timeit
import copy

from utils import utils, learner
from utils import PlantPathologyDataset as dataset

/home/braedon/.local/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 7.1.0. Several security issues (CVE-2020-11538, CVE-2020-10379, CVE-2020-10994, CVE-2020-10177) have been fixed in pillow 7.1.0 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [2]:
NUM_TRAIN = 750
NUM_VAL = 250
users = ('braedon', 'thomas', 'shangyi')
user = users[0]

# Needs to be the full path because of raytune
if user == users[1]:
    csv_file='C:\\Users\\tjtom\PycharmProjects\PlantPathology2021\\train.csv'
    root_dir='C:\\Users\\tjtom\PycharmProjects\PlantPathology2021\\train_images'
    mappings_dir='C:\\Users\\tjtom\PycharmProjects\PlantPathology2021\\labelMappings.csv'
elif user == users[0]:
    csv_file='/home/braedon/skole/cs175/PlantPathology2021/train.csv'
    root_dir='/home/braedon/skole/cs175/PlantPathology2021/train_images'
    mappings_dir='/home/braedon/skole/cs175/PlantPathology2021/labelMappings.csv'

plant_dataset = dataset.PlantPathologyDataset(csv_file=csv_file,
                                              root_dir=root_dir,
                                              mappings_dir=mappings_dir)
train_data = DataLoader(plant_dataset, batch_size=10, sampler= learner.ChunkSampler(NUM_TRAIN, 0))
validation_data = DataLoader(plant_dataset, batch_size=10, sampler=learner.ChunkSampler(NUM_VAL, NUM_TRAIN))

In [3]:
# Use GPU if available
if torch.cuda.is_available():
    print('Using GPU')
    dtype = torch.cuda.FloatTensor
    resources_per_trial = {'gpu': 1}
else:
    print('Using CPU')
    dtype = torch.FloatTensor
    resources_per_trial = {'cpu': 1}

Using GPU


### Testing resnet50

In [7]:
# Use GPU if available
if torch.cuda.is_available():
    resnet50 = models.resnet50(pretrained=True).cuda()
else:
    resnet50 = models.resnet50(pretrained=True)

loss_fn = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.RMSprop(resnet50.parameters(), lr=1e-3)

In [8]:
learner.train(resnet50, train_data, loss_fn, optimizer, dtype, 25, print_every=10)
learner.check_accuracy(resnet50, validation_data, dtype)

, 1, 3, 1])
torch.Size([10, 3, 400, 267]) tensor([ 4,  9,  1,  9,  7,  4, 11, 10, 10,  9])
torch.Size([10, 3, 400, 267]) tensor([ 4, 10,  6,  1,  9,  3,  3,  0,  4,  1])
torch.Size([10, 3, 400, 267]) tensor([ 4,  9,  0,  9,  0,  4,  9,  1, 10,  6])
torch.Size([10, 3, 400, 267]) tensor([0, 1, 6, 0, 1, 9, 1, 1, 3, 9])
torch.Size([10, 3, 400, 267]) tensor([9, 3, 6, 6, 1, 3, 9, 9, 9, 1])
torch.Size([10, 3, 400, 267]) tensor([4, 1, 3, 1, 3, 0, 1, 4, 9, 9])
torch.Size([10, 3, 400, 267]) tensor([ 1,  9,  4,  4,  6,  1,  9,  6,  3, 10])
t = 40, loss = 1.1094
torch.Size([10, 3, 400, 267]) tensor([6, 9, 9, 3, 3, 9, 4, 0, 3, 9])
torch.Size([10, 3, 400, 267]) tensor([7, 9, 9, 4, 4, 5, 9, 0, 3, 0])
torch.Size([10, 3, 400, 267]) tensor([3, 4, 9, 9, 0, 4, 1, 9, 9, 3])
torch.Size([10, 3, 400, 267]) tensor([3, 9, 9, 9, 9, 4, 1, 3, 4, 3])
torch.Size([10, 3, 400, 267]) tensor([ 6,  9,  6,  9,  9,  0,  9,  3,  1, 10])
torch.Size([10, 3, 400, 267]) tensor([9, 3, 6, 1, 4, 6, 9, 4, 1, 9])
torch.Size([10, 3, 

RuntimeError: CUDA out of memory. Tried to allocate 34.00 MiB (GPU 0; 5.80 GiB total capacity; 3.57 GiB already allocated; 60.44 MiB free; 3.68 GiB reserved in total by PyTorch)

In [4]:
# torch.save(resnet50, 'resnet50_1000_25.pickle')
resnet50 = torch.load('resnet50_1000_25.pickle')
learner.check_accuracy(resnet50, validation_data, dtype)

Checking accuracy on validation set
Got 109 / 250 correct (43.60)


0.436

In [5]:
import importlib
importlib.reload(learner)
learner.check_accuracy(resnet50, validation_data, dtype)

Checking accuracy on validation set
tensor([9, 0, 9, 6, 9, 9, 9, 9, 1, 9])
tensor([1, 9, 1, 9, 6, 9, 9, 9, 3, 9])
tensor([9, 9, 1, 1, 6, 9, 6, 3, 9, 9])
tensor([9, 9, 9, 9, 9, 9, 6, 9, 9, 9])
tensor([4, 4, 9, 9, 3, 9, 9, 9, 9, 3])
tensor([0, 3, 9, 9, 9, 9, 9, 9, 0, 9])
tensor([9, 3, 9, 9, 9, 3, 9, 9, 1, 9])
tensor([9, 9, 3, 9, 3, 9, 9, 9, 6, 9])
tensor([9, 9, 9, 1, 0, 9, 9, 0, 9, 9])
tensor([9, 9, 3, 9, 1, 9, 9, 9, 9, 9])
tensor([9, 1, 6, 3, 3, 9, 9, 9, 9, 0])
tensor([1, 0, 4, 3, 3, 9, 9, 0, 3, 9])
tensor([3, 9, 9, 9, 9, 9, 9, 9, 3, 3])
tensor([4, 9, 9, 3, 9, 9, 4, 9, 6, 9])


KeyboardInterrupt: 